In [282]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = 150

from requests import get

import warnings
warnings.filterwarnings('ignore')

import time
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

from bs4 import BeautifulSoup

In [9]:
df = pd.read_csv('nfl_data.csv')

In [13]:
df = df[df['PositionCategory'] == 'OFF']

In [15]:
df['Position'].unique()

array(['TE', 'OL', 'WR', nan, 'G', 'QB', 'C', 'OT', 'RB', 'FB'],
      dtype=object)

In [21]:
te = df[df['Position'] == 'TE']
wr = df[df['Position'] == 'WR']
qb = df[df['Position'] == 'QB']
bf = df[(df['Position'] == 'RB') | (df['Position'] == 'FB')]

In [22]:
te.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
0,0.0,10,1,2002,LAC,81,S.Alexander,TE,OFF,14,14,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,52.5,594.7,13.2,1.4,37.3,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.5,118.9,0,0,0.0,1.4,0.0,0.0,1.4,TE,0.0,5870629,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,S.Alexander,0.0,78,72,10,0,0,0,0,0,0,NaN,NaN,0,11.8,0.0,0.0,0.0,0.0,0.0,16.4,11.8,NaN,NaN,29,29,16.4,NaN,NaN,NaN,NaN,"[{'GameKey': '200211729', 'SeasonType': 1, 'Pl..."
2,2.0,18,1,2002,TEN,88,G.Kinne,TE,OFF,15,15,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,15.2,201.7,15.5,0.0,36.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,47.8,16.0,0.0,29.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.2,35.3,0,0,0.0,0.0,0.0,0.0,0.0,TE,0.0,5884471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G.Kinne,0.0,72,72,11,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,34,34,0.0,NaN,NaN,NaN,NaN,[]
14,14.0,91,1,2002,BUF,48,B.McDonnell,TE,OFF,6,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,TE,0.0,586

In [23]:
wr.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
4,4.0,20,1,2002,BAL,85,M.Wynn,WR,OFF,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,WR,0.0,5877964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M.Wynn,0.0,58,71,8,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3,3,0.0,NaN,NaN,NaN,NaN,[]
12,12.0,76,1,2002,NYG,87,D.Jones,WR,OFF,13,13,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,5.4,5.4,0.0,5.4,0,10.9,104.9,13.1,0.0,37.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,56.0,18.7,0.0,23.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,20.3,0,0,0.0,0.0,0.0,0.0,0.0,WR,0.0,5873221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D.Jones,0.0,70,63,7,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,23,23,0.0,NaN,NaN,NaN,NaN,[]
34,34.0,212,1,2002,NYJ,87,L.Coles,WR,OFF,16,16,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.2,45.5,8.8,0.0,24.5,0,103.8,1473.8,16.6,6.8,50.1,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,189.2,293.0,0,0,0.0,6.8,0.0,0.0,6.8,WR,0.0,5878005,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,

In [24]:
qb.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
11,11.0,65,1,2002,IND,7,B.Huard,QB,OFF,16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,QB,0.0,5874620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B.Huard,0.0,72,63,7,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,14,14,0.0,NaN,NaN,NaN,NaN,[]
31,31.0,198,1,2002,PHI,10,K.Detmer,QB,OFF,14,14,1,32.6,22.2,261.2,79.2,10.9,13.8,2.7,0.0,135.0,43.1,1.4,10.9,2.7,5.4,2.7,1.4,4.1,0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.2,27.2,0,0,0.0,1.4,0.0,0.0,1.4,QB,0.0,5864954,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,K.Detmer,0.0,68,68,12,0,0,0,0,0,0,NaN,NaN,0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.2,NaN,NaN,26,26,0.2,NaN,NaN,NaN,NaN,"[{'GameKey': '200211231', 'SeasonType': 1, 'Pl..."
33,33.0,211,1,2002,SF,11,B.Doman,QB,OFF,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,QB,0.0,5875277,0.0,0.0,0.0,0

In [25]:
bf.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
22,22.0,138,1,2002,NYG,29,D.Washington,RB,OFF,12,12,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.4,89.8,22.4,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.2,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,9.5,0.0,0.0,0.0,0.0,RB,0.0,5864554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D.Washington,0.0,71,63,8,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,23,23,0.0,NaN,NaN,NaN,NaN,[]
27,27.0,170,1,2002,BAL,31,J.Lewis,RB,OFF,16,16,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,359.1,1547.3,5.8,8.2,87.4,0,54.8,515.4,11.0,1.4,89.8,10.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.2,310.0,0,0,0.0,9.5,0.0,0.0,9.5,RB,0.0,5880987,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,J.Lewis,0.0,70,62,10,0,0,0,0,0,0,NaN,NaN,0,151.2,0.0,0.0,0.0,0.0,0.0,178.6,151.2,NaN,NaN,3,3,178.6,NaN,NaN,NaN,NaN,"[{'GameKey': '200210403', 'SeasonType': 1, 'Pl..."
36,36.0,224,1,2002,TEN,20,M.Green,RB,OFF,15,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.5,82.8,4.6,0.0,14.0,0,9.5,66.5,11.0,1.4,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,11.7,11.7,0.0,11.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.9,30.1,0,0,1.4,1.4,0.

In [222]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=11&mid1=1'

In [283]:
async def get_html(url, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                await page.mouse.wheel(0,15000)
                time.sleep(2)
                html = await page.content()
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [228]:
html = await get_html(url)

Fantasy Football | Yahoo! Sports


In [357]:
def clean_html(html, league_id, week, num_players):
    
    scores = []
    team_num = []
    team_name = []
    opp_score = []
    win_loss = []

    score_list = re.findall(r'>\d+\.\d+<', html)[-num_players:]
    for score in score_list:
        scores.append(score[1:-1])
        
    names = re.findall(r'14104/\d+\">[\w\' ]+<',html)[2:]
    for name in names:
        num = re.findall(r'/\d+', name)
        team_num.append(num[0][1:])
        name = re.findall(r'>.+<', name)
        team_name.append(name[0][1:-1]) 
    
    league_data = pd.DataFrame({'league':league_id , 'week':week,'team number':team_num,
                                'team name':team_name,'score':scores})
    
    for i in range(len(league_data)):
        if i % 2 == 0:
            opp_score.append(league_data.loc[(i+1), 'score'])
        else:
            opp_score.append(league_data.loc[(i-1), 'score'])
        
        if league_data.loc[i, 'score'] > opp_score[i]:
            win_loss.append(1)
        else:
            win_loss.append(0)
            
    league_data['opponent score'] = opp_score
    league_data['W/L'] = win_loss

            
    return league_data

In [358]:
week_data = clean_html(html, '14104', '11', 10)

In [359]:
week_data

,league,week,team number,team name,score,opponent score,W/L
0,14104,11,1,Bryan's Boss Team,117.12,107.52,1
1,14104,11,7,Bobby's Boss Team,107.52,117.12,0
2,14104,11,2,Freddie's Phantoms,104.96,98.22,0
3,14104,11,6,joe's Unbelievable Team,98.22,104.96,1
4,14104,11,3,Big Slum,98.20,131.80,1
5,14104,11,5,Colliers Wood Crows,131.80,98.20,0
6,14104,11,4,James's Marvelous Team,67.88,110.94,1
7,14104,11,10,Skidmarx,110.94,67.88,0
8,14104,11,8,Costly Errors,102.70,113.48,0
9,14104,11,9,SPERMINATING DOOM,113.48,102.70,1


In [284]:
fantasy_data = pd.DataFrame(columns=['league','week','team number','team name','score'])

In [ ]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=11&mid1=1'

In [364]:
async def get_data(fantasy_data, league_id, num_weeks, num_players):
    
    for week in range(1,num_weeks+1):
        url = f'https://football.fantasysports.yahoo.com/f1/{league_id}/matchup?week={week}&mid1=1'
        
        html = await get_html(url)
        
        week_data = clean_html(html, league_id, week, num_players)
        
        fantasy_data = pd.concat([fantasy_data, week_data])
        
        if week % 10 == 0: 
            print(f'Week {week} completed')
        
    return fantasy_data

In [366]:
fantasy_data = await get_data(fantasy_data, '14104', 15, 10)

Week 10 completed


In [367]:
fantasy_data

,league,week,team number,team name,score,opponent score,W/L
0,14104,1,1,Bryan's Boss Team,69.58,125.18,1.0
1,14104,1,2,Freddie's Phantoms,125.18,69.58,0.0
2,14104,1,3,Big Slum,90.32,77.82,1.0
3,14104,1,9,SPERMINATING DOOM,77.82,90.32,0.0
4,14104,1,4,James's Marvelous Team,133.42,132.20,1.0
...,...,...,...,...,...,...,...
5,14104,15,5,Colliers Wood Crows,131.80,98.20,0.0
6,14104,15,4,James's Marvelous Team,67.88,110.94,1.0
7,14104,15,10,Skidmarx,110.94,67.88,0.0
8,14104,15,8,Costly Errors,102.70,113.48,0.0


In [368]:
fantasy_data.to_csv('fantasy_data.csv')

In [308]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=15&mid1=1'

In [309]:
html = await get_html(url)

In [310]:
week_data = clean_html(html, '14104', 14, 10)

In [311]:
week_data

,league,week,team number,team name,score
0,14104,14,1,Bryan's Boss Team,117.12
1,14104,14,7,Bobby's Boss Team,107.52
2,14104,14,2,Freddie's Phantoms,104.96
3,14104,14,6,joe's Unbelievable Team,98.22
4,14104,14,3,Big Slum,98.20
5,14104,14,5,Colliers Wood Crows,131.80
6,14104,14,4,James's Marvelous Team,67.88
7,14104,14,10,Skidmarx,110.94
8,14104,14,8,Costly Errors,102.70
9,14104,14,9,SPERMINATING DOOM,113.48
